In [2]:
import numpy as np
from scipy.integrate import odeint
from scipy.signal import butter, filtfilt, find_peaks
from scipy.stats import gaussian_kde,ecdf
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
import time

In [3]:
def combined_model(I_net, M_net, c_net, z_net, tspan, init_conds):
    I_uncertainty = 0
    M_uncertainty = 0
    l_uncertainty = 0
    g_uncertainty = 0.0
    viscous_uncertainty = 0
    friction_uncertainty = 0
    noise_const = np.deg2rad(0.04, dtype=np.float128) #note that if you turn this to zero things appear to shit themselves
    Init_conds_uncertainty = [0, 0]

    I = np.float128(I_net) + ((2 * np.random.rand() - 1) * I_uncertainty)
    M = np.float128(M_net) + ((2 * np.random.rand() - 1) * M_uncertainty)
    l = 1 + ((2 * np.random.rand() - 1) * l_uncertainty)
    r = (.75 / 2) + ((2 * np.random.rand() - 1) * l_uncertainty)
    g = 9.7949 + ((2 * np.random.rand() - 1) * g_uncertainty)
    c = c_net + ((2 * np.random.rand() - 1) * viscous_uncertainty)
    z = z_net + ((2 * np.random.rand() - 1) * friction_uncertainty)
    ics = init_conds + ((2 * np.random.rand(2) - 1) * Init_conds_uncertainty)

    paramsTrue = [I, M, g, r, l, c, z]

    def combined_system(T, t):
        I, M, g, r, l, c, z = paramsTrue
        tdot1 = T[1]
        tdot2 = ((-z * np.sign(T[1])) - (c * T[1]) - ((M * g * r ** 2 / l) * T[0])) / I
        return [tdot1, tdot2]

    theta_m = odeint(combined_system, ics, tspan)
    noise = noise_const * (2 * np.random.rand(len(theta_m[:, 0])) - 1)

    theta = theta_m + noise[:, None]
    return tspan, theta


In [8]:
I_names = [
    'I_x', 'I_xy_x_rot', 'I_xz_x_rot', 
    'I_y', 'I_xy_y_rot', 'I_yz_y_rot', 
    'I_z', 'I_xz_z_rot', 'I_yz_z_rot'
]

I_values = {}

x_dim = 3 #m
y_dim = 4 #m
z_dim = 5 #m
mass = 5 #kg
phi_xy = np.deg2rad(4) #rad
phi_xz = np.deg2rad(6) #rad
phi_yz = np.deg2rad(3) #rad

theta = np.deg2rad(30) #rad

#principle Axis Moments of Inertia
I_x = (1/12)*mass*(y_dim**2 + z_dim**2)
I_y = (1/12)*mass*(x_dim**2 + z_dim**2)
I_z = (1/12)*mass*(x_dim**2 + y_dim**2) 

#derived units
mu_xy = np.mean([I_x,I_y])
r_xy = (I_x-I_y)/2
mu_xz = np.mean([I_x,I_z])
r_xz = (I_x-I_z)/2
mu_yz = np.mean([I_y,I_z])
r_yz = (I_y-I_z)/2

#measured values
I_values[I_x] = np.mean([mu_xy+r_xy*np.cos(2*phi_xy),mu_xz+r_xz*np.cos(2*phi_xz)])
print(mu_xy+r_xy*np.cos(2*phi_xy))
print(mu_xz+r_xz*np.cos(2*phi_xz))
I_values[I_y] = np.mean([mu_xy-r_xy*np.cos(2*phi_xy),mu_yz+r_yz*np.cos(2*phi_yz)])
I_values[I_z] = np.mean([mu_xz-r_xz*np.cos(2*phi_xz),mu_yz-r_yz*np.cos(2*phi_yz)])
I_values[I_xy_x_rot] = mu_xy+np.cos(2*(theta+phi_xy))
I_values[I_xy_y_rot] = mu_xy-np.cos(2*(theta+phi_xy))
I_values[I_xz_x_rot] = mu_xz+np.cos(2*(theta+phi_xz))
I_values[I_xz_z_rot] = mu_xz-np.cos(2*(theta+phi_xz))
I_values[I_yz_y_rot] = mu_yz+np.cos(2*(theta+phi_yz))
I_values[I_yz_z_rot] = mu_yz-np.cos(2*(theta+phi_yz))

#printing tensors
I_principle = [[I_x,0,0],[0,I_y,0],[0,0,I_z]]
I_actual = [[],[],[]]

15.878236925764273
12.083333333333334


NameError: name 'I_xy_x_rot' is not defined